# Originality Algo

## Algorithm to Automate Originality Scoring

### Import Packages

In [44]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import openpyxl
import xlsxwriter

from nltk.stem import WordNetLemmatizer
import string

from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.feature_extraction.text import CountVectorizer

import matplotlib.pyplot as plt
from yellowbrick.cluster import KElbowVisualizer

from nltk.cluster.kmeans import KMeansClusterer

### Put Data from Excel Sheet into Dataframes

In [2]:
# individual df's for each sheet

# when on pc
data_test_cup = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_cup_semdis.csv")
data_test_key = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_key_semdis.csv")
data_test_rope = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_rope_semdis.csv")
data_test_brick = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_brick_semdis.csv")
data_test_chair = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_chair_semdis.csv")
data_test_pencil = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_pencil_semdis.csv")
data_test_shoe = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_shoe_semdis.csv")

# when on mac
# data_test_cup = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_cup_semdis.csv.xlsx")
# data_test_key = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_key_semdis.csv.xlsx")
# data_test_rope = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_rope_semdis.csv.xlsx")
# data_test_brick = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_brick_semdis.csv.xlsx")
# data_test_chair = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_chair_semdis.csv.xlsx")
# data_test_pencil = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_pencil_semdis.csv.xlsx")
# data_test_shoe = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_shoe_semdis.csv.xlsx")

In [3]:
# individual df's for each sheet

# when on pc
data_official_cup = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_cup.csv")
data_official_key = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_key.csv")
data_official_rope = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_rope.csv")
data_official_brick = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_brick.csv")
data_official_chair = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_chair.csv")
data_official_pencil = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_pencil.csv")
data_official_shoe = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_shoe.csv")
data_official_box = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_box.csv")

# when on mac
# data_official_cup = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_cup.csv")
# data_official_key = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_key.csv")
# data_official_rope = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_rope.csv")
# data_official_brick = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_brick.csv")
# data_official_chair = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_chair.csv")
# data_official_pencil = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_pencil.csv")
# data_official_shoe = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_shoe.csv")
# data_official_box = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/official/official_csvs/autdata_official_box.csv")

### Preprocessing

In [4]:
# spacy stop words
stopwords_spacy = STOP_WORDS

In [5]:
stopwords_edited = list(stopwords_spacy)
stopwords_edited.append("thing")
stopwords_edited.append("use")
stopwords_edited.append("things")

In [6]:
# method to clean the responses
def process_text(text, stopwords_list, join_list):
    # tokenize text, lemmanize words, removing punctuation, remove stop words, lowercase all words
    
    text = re.sub("/|-"," ", text)
    text = text.translate(str.maketrans('','',string.punctuation))
    tokens = word_tokenize(text)

    tokens = [w.lower() for w in tokens]
    
    tokens = [word for word in tokens if word not in stopwords_list]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
#         stemmer = PorterStemmer()
#         tokens = [stemmer.stem(t) for t in tokens]

    if join_list:
        tokens = ' '.join(tokens)
 
    return tokens

### General Functions

In [7]:
# method to get a list of participants
def get_id_list(df):
    id_list = df['id'].unique()
    id_list = sorted(id_list)
    return id_list

In [8]:
# method to add a new column
# new column are cleaned responses
def get_cleaned_responses(df, stopwords_list, join_list):
    # id_df = df[df.id == id]
    df_processed = df.copy(deep=True)
    responses = df['response'].tolist()

    # make list of processed responses
    for response in range(len(responses)):
        responses[response] = process_text(responses[response], stopwords_list, join_list)

    # add list as column in df
    df_processed['response_processed'] = responses

    return df_processed

## Originality Algo 1
### Term Frequency Only

In [9]:
# create term frequecy dictionary
def get_tf_dict(responses):
    cv = CountVectorizer()   
    cv_fit = cv.fit_transform(responses)    
    word_list = cv.get_feature_names()

    # [0] here to get a 1d-array for iteration by the zip function 
    count_list = np.asarray(cv_fit.sum(axis=0))[0]

    tf_dict = dict(zip(word_list, count_list))
    
    total_words = 0
    for key in tf_dict:
        total_words = total_words + tf_dict[key]
        
    for key in tf_dict:
        tf_dict[key] = np.log(1 + tf_dict[key]/total_words)
        
    tf_dict = dict(sorted(tf_dict.items(), key=lambda item: item[1], reverse=True))
                
    return tf_dict

In [10]:
# get sum of term frequency in a sentence
def get_tf_sum(tf_dict, response):
    tf_sum = 0
    for term in range(len(response)):
        tf_sum = tf_sum + tf_dict[response[term]]
        
    if tf_sum == 0:
        tf_sum = np.nan
        
    return tf_sum

In [11]:
# get parallel list to dataframe for the tf sum
def get_tf_sum_list(tf_dict, responses):
    tf_sum_list = []
    
    for response in responses:
        tf_sum_list.append(get_tf_sum(tf_dict, response))
            
    return tf_sum_list

In [12]:
# method to calculate a phrases term frequency
# return df
def get_originality_tf_only(df, stopwords_list, join_list):
    originality_rating_df = get_cleaned_responses(df, stopwords_list, join_list)
    responses = originality_rating_df['response_processed'].tolist()
    responses_tokenized = [item for sublist in responses for item in sublist]

    tf_dict = get_tf_dict(responses_tokenized)
    
    tf_sum_list = get_tf_sum_list(tf_dict, responses)

    originality_rating_df['tf_sum'] = tf_sum_list
        
    return originality_rating_df

In [13]:
get_originality_tf_only(data_test_brick, stopwords_edited, False)

,id,response,response_nofill,item,item_nofill,SemDis_cbowukwacsubtitle_nf_m,response_processed,tf_sum
0,1476,as a foundation,foundation,Brick,brick,0.77417,[foundation],0.011834
1,1476,as a weapon,weapon,Brick,brick,0.85011,[weapon],0.068993
2,1476,as a bowl,bowl,Brick,brick,0.82926,[bowl],0.011834
3,1718,build a house,build house,Brick,brick,0.93854,"[build, house]",0.159602
4,1718,break a window,break window,Brick,brick,0.96992,"[break, window]",0.070051
5,1718,line a fireplace,line fireplace,Brick,brick,0.99511,"[line, fireplace]",0.035365
6,1718,line a road,line road,Brick,brick,0.99395,"[line, road]",0.047061
7,1691,build house,build house,Brick,brick,0.93854,"[build, house]",0.159602
8,1691,break window,break window,Brick,brick,0.96992,"[break, window]",0.070051
9,1691,weight for workout,weight workout,Brick,brick,1.05438,"[weight, workout]",0.069654


## Originality Algo 2
### Counter Vectorizer + clustering

In [20]:
# use count vectorizer to cluster phrases
# return clustering df
def get_counts_vector(num_clusters, responses):
    # initialize CountVectorizer object
    count_vectorizer = CountVectorizer()
    # vectorize the phrases
    word_count = count_vectorizer.fit_transform(responses)
    
    # elbow method to visualize and find out how many clusters to use
#     visualizer = KElbowVisualizer(KMeans(), k=(10,35), timings=False)
#     visualizer.fit(word_count.toarray())       
#     visualizer.show()

    # nltk kmeans cosine distance implementation
    number_of_clusters = num_clusters
    kmeans = KMeansClusterer(number_of_clusters, repeats=25, distance=nltk.cluster.util.cosine_distance, avoid_empty_clusters=True)
    assigned_clusters = kmeans.cluster(word_count.toarray(), assign_clusters=True)
        
    # cluster results scikit-learn
    results = pd.DataFrame()
    results['text'] = responses
#     results['category'] = kmeans.labels_
    results['category'] = assigned_clusters
    
    # create dictionary to organize the clusters with their respective phrases
    results_dict = {k: g["text"].tolist() for k,g in results.groupby("category")}
    
    # df of the clusters and the 
    clusters_df = pd.DataFrame(list(results_dict.items()),columns = ['category','responses']) 
    
    return clusters_df

In [21]:
# calculate originality score for a phrase based on number of responses in a cluster
# calculate the rarity of a phrase
def get_clustered_originality_score(originality_rating_df, num_clusters, column):
    # get cluster df
    clusters_df = get_counts_vector(num_clusters, originality_rating_df['response_processed'].tolist())

    # create dictionary out of cluster df
    clusters = dict(zip(clusters_df.category, clusters_df.responses))
        
    # initialize empty dictionary to store the score for a category
    clusters_scores = dict.fromkeys(clusters)
    
    # initialize empty dictionary to store 
    # will store 0 or 1, 1 if only 1 response in cluster
    clusters_rarity = dict.fromkeys(clusters)
    
    # get the average cosine distance for a cluster
    for key in clusters:
        score = 1.0
        score = score - (len(clusters[key])/len(originality_rating_df.index))
        clusters_scores[key] = score
        if (len(clusters[key]) == 1):
            clusters_rarity[key] = 1
        else:
            clusters_rarity[key] = 0
        
    # create dictionary to store a phrase and its new originality score 
    # new score is the average of the responses in one cluster
    phrase_scores_dict = {}
    # create dictionary to store a phrase and its rarity 
    # will be 0 or 1, 1 if only response in that cluster
    phrase_rarity_dict = {}
    for key in clusters:
        for phrase in clusters[key]:
            phrase_scores_dict[phrase] = clusters_scores[key]
            phrase_rarity_dict[phrase] = clusters_rarity[key]
            
    # make a list that matches the one in the current dataframe
    # return list to be added to dataframe
    df_phrases_scores_list = [] 
    # make a list that matches the one in the current dataframe
    # return list to be added to dataframe
    df_phrases_rarity_list = []
    for phrase in originality_rating_df['response_processed'].tolist():
        df_phrases_scores_list.append(phrase_scores_dict[phrase])
        df_phrases_rarity_list.append(phrase_rarity_dict[phrase])
    
    # comment out to see the clusters df
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(clusters_df)
    
    print(clusters_scores)
                
    return (df_phrases_scores_list, df_phrases_rarity_list)

In [22]:
# create df of each participants originality count score, the amount of 
def get_originality_score_df(originality_df):
    # get id list and create empty dict
    id_list = get_id_list(originality_df)
    participants_originality = {k: 0 for k in id_list}
    # add the number of unique responses aka clusters with only 1 response
    for participant in id_list:
        id_df = originality_df[originality_df.id == participant]
        participants_originality[participant] = id_df['originality'].sum()
        
    return pd.DataFrame(participants_originality.items(), columns=['id', 'originality'])

In [28]:
# calculate the originality freq and counts for a response and participant respectively
def get_originality_count_vectorizer(df, stopwords_list, num_clusters, join_list):
    originality_rating_df = get_cleaned_responses(df, stopwords_list, join_list)
    originality_rating_df = originality_rating_df[originality_rating_df.astype(str)['response_processed'] != '']
    responses = originality_rating_df['response_processed'].tolist()
            
    originality_results = get_clustered_originality_score(originality_rating_df, num_clusters, responses)
    # add frequency column to results df
    originality_rating_df['cluster_freq'] = originality_results[0]
    # add 1/0 column to show if that phrase was by itself in a cluster
    originality_rating_df['originality'] = originality_results[1]
    
    originality_rating_df['freq'] = (1 - originality_rating_df['cluster_freq'])
    originality_rating_df['t_freq'] = (.05/(.05 + originality_rating_df['freq']))**2
    
    # get the original phrases counts df
    originality_scores = get_originality_score_df(originality_rating_df)

    return (originality_rating_df, originality_scores)

In [29]:
originality_results = get_originality_count_vectorizer(data_official_rope, stopwords_edited, 80, True)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(originality_results)

    category                                          responses
0          0               [sailing, sailing, sailing, sailing]
1          1               [jewelry, jewelry, jewelry, jewelry]
2          2                [noose, noose, noose, noose, noose]
3          3  [hang, hang, hang, hang, hang, hang, hang, han...
4          4                                            [shred]
5          5                                      [measurement]
6          7               [whip, whip, whip, whip, whip, whip]
7          8                                         [twisting]
8         10  [climb, climb, climb, climb, climb, climb, cli...
9         12               [working, working, working, working]
10        13                   [weapon, weapon, weapon, weapon]
11        14  [tie, tie, tie, tie, tie, tie, tie, tie, tie, ...
12        15  [lasso, lasso, lasso, lasso, lasso, lasso, las...
13        17   [pull, pull, pull, pull, pull, pull, pull, pull]
14        18  [climbing, climbing, climb

## Algo Results with Official Data

In [31]:
prompts_list = ['box', 'brick', 'chair', 'cup', 'key', 'pencil', 'rope', 'shoe']
data_list = [data_official_box, data_official_brick, data_official_chair, data_official_cup, data_official_key, data_official_pencil, data_official_rope, data_official_shoe]

In [32]:
# calculate originality scores from method
def get_originality_scores(data_list, num_clusters):
    results_list = []
    for i in range(len(data_list)):
        # get originality results for each dataset
        results = get_originality_count_vectorizer(data_list[i], stopwords_edited, num_clusters, True, True)
        # add originality results to a list
        results_list.append(results)
        
    # return list
    return results_list

In [33]:
originality_results_list = get_originality_scores(data_list, 80)

    category                                          responses
0          0  [store, store, store, store, store, store, sto...
1          1                               [organize, organize]
2          2                               [standing, standing]
3          3                                         [stacking]
4          4                                            [dance]
5          5                                 [mailing, mailing]
6          7                                           [return]
7          8                                            [float]
8          9                        [shelter, shelter, shelter]
9         11  [cardboard, cardboard, cardboard, cardboard, c...
10        12                                 [support, support]
11        13                              [stack, stack, stack]
12        14                           [moving, moving, moving]
13        16                                            [brown]
14        18                            

    category                                          responses
0          0  [sleeping, sleeping, sleeping, sleeping, sleep...
1          1  [sit, sit, sit, sit, sit, sit, sit, sit, sit, ...
2          2  [sitting, sitting, sitting, sitting, sitting, ...
3          5                [sleep, sleep, sleep, sleep, sleep]
4          8                     [decorate, decorate, decorate]
5          9                              [stool, stool, stool]
6         11  [weapon, weapon, weapon, weapon, weapon, weapo...
7         12                                    [hit, hit, hit]
8         15                                       [seat, seat]
9         18                [throw, throw, throw, throw, throw]
10        21                        [storage, storage, storage]
11        23                                       [play, play]
12        24                                 [support, support]
13        26                                             [game]
14        27                            

    category                                          responses
0          0                                          [mailbox]
1          1                                       [kill, kill]
2          2  [unlock, unlock, unlock, unlock, unlock, unloc...
3          3                                           [magnet]
4          4                                          [melting]
5          5                                       [persuasion]
6          8                     [lock, lock, lock, lock, lock]
7          9  [weapon, weapon, weapon, weapon, weapon, weapo...
8         11  [necklace, necklace, necklace, necklace, neckl...
9         12                                     [house, house]
10        13  [jewelry, piece jewelry, jewelry, jewelry, jew...
11        14  [scratch sidewalk, scratch, scratch, scratch, ...
12        15  [decoration, decoration, decoration art, decor...
13        16        [screwdriver, screwdriver, tip screwdriver]
14        17                       [musi

    category                                          responses
0          0                                         [whipping]
1          1                                          [squeeze]
2          2               [working, working, working, working]
3          3                                          [shaking]
4          4   [knot, knot, knot, knot, knot, knot, knot, knot]
5          5  [lasso, lasso, lasso, lasso, lasso, lasso, las...
6          7                                       [jump, jump]
7          8  [tie, tie, tie, tie, tie, tie, tie, tie, tie, ...
8         10                                             [fish]
9         11                                           [fixing]
10        12                                         [twisting]
11        14                                            [sport]
12        15   [pull, pull, pull, pull, pull, pull, pull, pull]
13        16                                    [hit, hit, hit]
14        19               [sailing, sai

In [35]:
originality_results_list[0][0]

,id,item,response,rating,creativity,response_processed,cluster_freq,originality,freq,t_freq
0,1094,Box,to store things,1,1,store,0.963719,0,0.036281,0.335821
1,1094,Box,to build a fort,3,4,build fort,0.979592,0,0.020408,0.504306
2,1094,Box,as an umbrella,3,4,umbrella,0.741497,0,0.258503,0.026268
3,1094,Box,to catch a bunny,3,4,catch bunny,0.741497,0,0.258503,0.026268
4,1093,Box,standing on,2,2,standing,0.995465,0,0.004535,0.840595
...,...,...,...,...,...,...,...,...,...,...
442,1372,Box,trap unwanted bugs or animals,3,3,trap unwanted bug animal,0.986395,0,0.013605,0.617947
444,1525,Box,cut it open and use it as a floor mat,2,4,cut open floor mat,0.990930,0,0.009070,0.716476
445,1525,Box,use it as a support to put things on,2,3,support,0.995465,0,0.004535,0.840595
446,1525,Box,let a cat play in it,3,3,let cat play,0.979592,0,0.020408,0.504306


## Write Originality Results into Excel

In [38]:
# write originality freq results into excel sheet
def write_originality_results_freq(results):
    writer = pd.ExcelWriter('originality_methods_results_freqs_071521.xlsx', engine='xlsxwriter')
    workbook = writer.book
    
    for i in range(len(prompts_list)):
        worksheet = workbook.add_worksheet(prompts_list[i])
        writer.sheets[prompts_list[i]] = worksheet
        results[i][0].to_excel(writer, sheet_name = prompts_list[i], startrow = 0, startcol = 0, index = False)
            
    writer.save()

In [39]:
# write originality counts results into excel sheet
def write_originality_results_counts(results):
    writer = pd.ExcelWriter('originality_methods_results_counts_071521.xlsx', engine='xlsxwriter')
    workbook = writer.book
    
    for i in range(len(prompts_list)):
        worksheet = workbook.add_worksheet(prompts_list[i])
        writer.sheets[prompts_list[i]] = worksheet
        results[i][1].to_excel(writer, sheet_name = prompts_list[i], startrow = 0, startcol = 0, index = False)
            
    writer.save()

In [40]:
write_originality_results_freq(originality_results_list)

In [41]:
write_originality_results_counts(originality_results_list)

Algo Design Brainstorming:
* Algo Design w/o scikit-learn
    * asdlfjakl;fjka
    * asdfasf

* Algo Design w/ scikit-learn

To Do List
- [x] write basic algorithm, sum of term frequency
    - frequency based on all phrases
- [x] write tfidf clustering method 
    - repurpose for flexibility
- [x] write counter vectorizer clustering method
- [x] write method to score responses by frequency
    - number of responses in a cluster/number of total responses subtracted from 1
    - subtract from 1 to get the reverse mapping
- [x] experiment with a greater cluster size
- [x] update stop words list to include "use" and "thing"
- [ ] figure out how many times to run kmeans
    - cross validation
    - then averaging the results of all the iterations
